<h1>Graph implementation</h1>


In [1]:
class Node:
    def __init__(self, name):
        self.name = name
        self.edges = []
        
    def add_edge(self, node, weight):
        self.edges.append((node, weight))

    def remove_edge(self, node):
        for edge in self.edges:
            if edge[0] == node:
                self.edges.remove(edge)
                break

class Graph:
    def __init__(self):
        self.vertices = {}
        
    def add_node(self, node):
        self.vertices[node.name] = node
        
    def add_edge(self, node1, node2, weight):
        node1.add_edge(node2, weight)
        node2.add_edge(node1, weight)
        
    def remove_node(self, node):
        del self.vertices[node.name]
        for vertex in self.vertices.values():
            vertex.remove_edge(node)
    
    def remove_edge(self, node1, node2):
        node1.remove_edge(node2)
        node2.remove_edge(node1)
        
    def search(self, item):
        for node in self.vertices.values():
            if node.name == item:
                return True
        return False
    
# graph = Graph()

# a = Node('A')
# b = Node('B')
# c = Node('C')
# d = Node('D')
# e = Node('E')
# f = Node('F')
# g = Node('G')

# graph.add_node(a)
# graph.add_node(b)
# graph.add_node(c)
# graph.add_node(d)
# graph.add_node(e)
# graph.add_node(f)
# graph.add_node(g)

# graph.add_edge(a, b, 2)
# graph.add_edge(a, c, 5)
# graph.add_edge(b, c, 6)
# graph.add_edge(b, d, 1)
# graph.add_edge(b, e, 3)
# graph.add_edge(c, f, 8)
# graph.add_edge(d, e, 4)
# graph.add_edge(e, g, 9)
# graph.add_edge(f, g, 7)

# for node in graph.vertices.values():
#     for edge in node.edges:
#         print(f"{node.name} {edge[0].name} {edge[1]}")


# Loading graph from a file containing the cities 


In [2]:
def load_graph_from_file(filename):
    graph = Graph()
    with open(filename, 'r') as f:
        for line in f:
            node1, node2, weight = line.strip().split()
            if node1 not in graph.vertices:
                graph.add_node(Node(node1))
            if node2 not in graph.vertices:
                graph.add_node(Node(node2))
            graph.add_edge(graph.vertices[node1], graph.vertices[node2], int(weight))
    return graph

# BFS Implementation 

In [22]:
def bfs(graph, start, end):
    queue = [start]
    visited = [start]
    paths = {start: [start]}

    while queue:
        vertex = queue.pop(0)
        for neighbor in graph.vertices[vertex].edges:
            if neighbor[0].name not in visited:
                visited.append(neighbor[0].name)
                queue.append(neighbor[0].name)
                paths[neighbor[0].name] = paths[vertex] + [neighbor[0].name]
                if neighbor[0].name == end:
                    return paths[end]

    return None


<h1>Loaded Graph</h1>

In [3]:
graph = load_graph_from_file('cities.txt')


In [296]:
# bfs(graph,"Zerind","Vaslui")

# DFS Implementation

In [80]:
def dfs(graph, start, end):
    stack = [start]
    visited = [start]
    paths = {start: [start]}

    while stack:
        vertex = stack.pop()
        for neighbor in graph.vertices[vertex].edges:
            if neighbor[0].name not in visited:
                visited.append(neighbor[0].name)
                stack.append(neighbor[0].name)
                paths[neighbor[0].name] = paths[vertex] + [neighbor[0].name]
                if neighbor[0].name == end:
                    return paths[end]

    return None


In [25]:
dfs(graph,"Arad","Fagaras")

['Arad',
 'Timisoara',
 'Lugoj',
 'Mehadia',
 'Drobeta',
 'Craiova',
 'Pitesti',
 'Bucharest',
 'Fagaras']

<h1>UCS Implementation</h1>

In [105]:
import queue

def ucs(graph, start, end):
    pq = queue.PriorityQueue()
    pq.put((0, [start]))
    visited = set()
    
    while not pq.empty():
        (cost, path) = pq.get()
        node = path[-1]
        if node == end:
            return path
        if node in visited:
            continue
        visited.add(node)
        for neighbor, weight in graph.vertices[node].edges:
            if neighbor.name not in visited:
                new_cost = cost + weight
                new_path = path + [neighbor.name]
                pq.put((new_cost, new_path))
    
    return None


In [27]:
ucs(graph,"Oradea","Bucharest")


['Oradea', 'Sibiu', 'Rimnicu', 'Pitesti', 'Bucharest']

<h1>Iterative Deepening Implementation</h1>

In [115]:
def iterative_deepening(graph, start, end):
    depth = 0
    while True:
        result = depth_limit_search(graph, start, end, depth)
        if result is not None:
            return result
        depth += 1

def depth_limit_search(graph, start, end, depth):
    stack = [(start, 0)]
    visited = [start]
    paths = {start: [start]}

    while stack:
        vertex, level = stack.pop()
        if level <= depth:
            for neighbor in graph.vertices[vertex].edges:
                if neighbor[0].name not in visited:
                    visited.append(neighbor[0].name)
                    stack.append((neighbor[0].name, level+1))
                    paths[neighbor[0].name] = paths[vertex] + [neighbor[0].name]
                    if neighbor[0].name == end:
                        return paths[end]

    return None


In [29]:
iterative_deepening(graph,"Arad","Eforie")

['Arad', 'Sibiu', 'Fagaras', 'Bucharest', 'Urziceni', 'Hirsova', 'Eforie']

<h1>Bidirectional Search Implementation</h1>

In [123]:
def b_f_s(graph, start, end, visited, paths, queue):
    vertex = queue.pop(0)
    for neighbor in graph.vertices[vertex].edges:
        if neighbor[0].name not in visited:
            visited.add(neighbor[0].name)
            queue.append(neighbor[0].name)
            paths[neighbor[0].name] = paths[vertex] + [neighbor[0].name]
            if neighbor[0].name == end:
                return paths[end], True
    return None, False

def bidirectional_bfs(graph, start, end):
    start_queue = [start]
    start_visited = set([start])
    start_paths = {start: [start]}

    end_queue = [end]
    end_visited = set([end])
    end_paths = {end: [end]}

    while start_queue and end_queue:
        path, found = b_f_s(graph, start, end, start_visited, start_paths, start_queue)
        if found:
            return path
        path, found = b_f_s(graph, end, start, end_visited, end_paths, end_queue)
        if found:
            return path[::-1]
    return None


In [31]:
bidirectional_bfs(graph,"Arad","Eforie")

['Arad', 'Sibiu', 'Fagaras', 'Bucharest', 'Urziceni', 'Hirsova', 'Eforie']

<h1>Heuristic Function</h1>

<h3>Loading the coordinates(latitude and longitude) of the cities</h3>

In [32]:
def read_coordinates(filename):
    cities_coordinate = {}
    with open(filename) as f:
        for line in f:
            values = line.strip().split()
            city = values[0]
            latitude = values[1]
            longitude = values[2]
            cities_coordinate[city] = (float(latitude), float(longitude))
    return cities_coordinate

city_coordinates = read_coordinates('coordinates.txt')

<h3>Geodesic distance calculator using the Haversine fomula </h3>

In [33]:
from math import radians, sin, cos, sqrt, atan2

def distance(coordinates, city1, city2):
    lat1, lon1 = coordinates[city1]
    lat2, lon2 = coordinates[city2]
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c #6371 is the raidus of earth
    
    return distance*0.62137119    #returning the distance in miles(1Km == 0.62137119mile)


In [34]:
def heuristic(city, destination):
    return distance(city_coordinates, city, destination)
    

In [35]:
print(heuristic("Arad", 'Bucharest'))

261.07177160527914


<h1>Greedy Search Implementation</h1>

In [265]:
import queue

def greedy_search(graph, start, end):
    pq = queue.PriorityQueue()
    pq.put((0, start))
    visited = set()
    paths = {start: [start]}


    while not pq.empty():
        current_cost, current_node = pq.get()
        if current_node == end:
            return paths[current_node]

        if current_node not in visited:
            visited.add(current_node)
            for neighbor in graph.vertices[current_node].edges:
                neighbor_node, edge_cost = neighbor[0].name, neighbor[1]
                if neighbor_node not in visited:
                    priority = mst_heuristic(neighbor_node, end, graph)
                    pq.put((priority, neighbor_node))
                    if neighbor_node not in paths:
                        paths[neighbor_node] = paths[current_node] + [neighbor_node]

    return None


In [266]:
# greedy_search(graph, 'A', 'F')

['A', 'C', 'F']

<h1>A* Search Implementation</h1>

In [328]:
import queue

def aStarSearch(graph, start, end):
    pq = queue.PriorityQueue()
    pq.put((0 + new_heuristic(graph, start, end), [start]))
    visited = set()
    
    while not pq.empty():
        (cost, path) = pq.get()
        node = path[-1]
        if node == end:
            return path
        if node in visited:
            continue
        visited.add(node)
        for neighbor, weight in graph.vertices[node].edges:
            if neighbor.name not in visited:
                new_cost = cost - new_heuristic(graph, start, end) + weight + new_heuristic(graph, neighbor.name, end)
                new_path = path + [neighbor.name]
                pq.put((new_cost, new_path))
    
    return None

    

In [263]:
# aStarSearch(graph, 'Oradea', 'Eforie')

['A', 'B', 'E', 'G']

<h2>Path cost calculator for evaluation<h2>

In [42]:
# We can use this find the costs of the paths returned by the algorithms
def path_cost_calculator(graph, path):
        total_cost = 0
        for i in range(len(path)-1):
            node1 = graph.vertices[path[i]]
            node2 = graph.vertices[path[i+1]]
            edge_weight = None
            for edge in node1.edges:
                if edge[0] == node2:
                    edge_weight = edge[1]
                    break
            total_cost += edge_weight
        return total_cost

In [41]:
path_cost_calculator(graph,['Zerind', 'Oradea', 'Sibiu', 'Fagaras', 'Bucharest', 'Urziceni', 'Vaslui'])

759

<h2>Experiment runner function</h2>

In [1]:
#THis function can help us run the experiment repeatitively, autonomusly

import timeit

def algorithm_runner(algorithm, cities, num_runs, graph):
  
    total_time = 0
    city1, city2 = cities[0], cities[1]
    time_taken_list = []

    for i in range(num_runs):
        time_taken = 0
        def inner_func():
            nonlocal time_taken
            time_taken = algorithm(graph, city1, city2)
        
        time_taken = timeit.timeit(inner_func, number=1)
        time_taken_list.append(round(time_taken, 5))
        total_time += round(time_taken, 5)

    avg_time = total_time / num_runs
    time_taken_list.append(round(avg_time, 5))
    
    #we added the cost in the time_taken_list, it can easily be identified that the last element is the cost
    #and the element next to the last element is the average of the time records in Seconds
    path = algorithm(graph, city1, city2)
    path_cost = path_cost_calculator(graph, path)
    time_taken_list.append(path_cost)
    
    return time_taken_list


In [2]:
# algorithm_runner(greedy_search, ['Zerind', 'Sibiu'], 10, graph)

<h1>A simple Table column writer function </h1>
<h4>This simple function uses the python docx lib. and helps us simplify tasks by writing table cells in MS-WORD</h4>

In [3]:
import docx
from docx import Document
# It takes in input the time_list which contains the results of the 10 experiments and the avg time

def table_writer(time_list, docname):
    document = Document(docname)
    table = document.tables[16]
    count = 1
    for data in time_list:
        table.cell(count, 2).text = str(data)
        count += 1
        
    document.save(docname)

##### NOTE: we varied the table indexes manually to fill the table.


In [4]:
# Running the Algorithms
time_list = alorithm_runner(aStarSearch, ['Zerind', 'Timisoara'], 10, graph)
# table_writer(time_list, 'Report.docx' 

NameError: name 'alorithm_runner' is not defined

<h1>Random Graph generation</h1>

<h2>Creating Erdős–Rényi model graphs with fixed probabilities</h2>

In [ ]:
import random
def generate_random_graph(n, p):
    nodes = [Node(chr(i)) for i in range(65, 65 + n)]
    
    graph = Graph()
    for node in nodes:
        graph.add_node(node)
    # Connect nodes randomly based on probability p
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < p:
                graph.add_edge(nodes[i], nodes[j], random.randint(1, 10))
                #The edge weights are randomly generated between 1 and 10
    return graph

<h4>Generating 16 different graphs with different probabilty and number of nodes and random edge weights</h4>

In [ ]:
node_list = [10, 20, 30, 40]

# List of edge probabilities
probability_list = [0.2, 0.4, 0.6, 0.8]

# Generating 16 random graphs
graphList = []

for n in node_list:
    for p in probability_list:
        graphList.append(generate_random_graph(n, p))

<h4>Randomly selecting 5 nodes and forming pairs between them, resulting in C(5, 2)=10 pairs for each graphs</h4>

In [ ]:
import random
from itertools import combinations
node_pairs = [] #node_pairs is list of list of tuples

for graph in graphList:
    nodes = list(graph.vertices.values())
    selected_nodes = random.sample(nodes, 5)
    # create unique node pairs
    pairs = list(combinations(selected_nodes, 2))
    node_pairs.append(pairs)


<h4>Validating the pairs generated(cheking if path exists between them)</h4>

In [ ]:
from typing import List
def validate_pairs(graph_list: List[Graph], node_pairs: List[List[tuple]]) -> List[List[tuple]]:
    valid_node_pairs = []
    for i, graph_pairs in enumerate(node_pairs):
        valid_pairs = []
        for pair in graph_pairs:
            start_node, end_node = pair
            path = bfs(graph_list[i], start_node.name, end_node.name)
            if path:
                valid_pairs.append(pair)
        valid_node_pairs.append(valid_pairs)
    return valid_node_pairs
valid_node_pairs = validate_pairs(graphList, node_pairs)

<h3>Designing new Heuristic function for the randomly generated graphs </h3>

In [ ]:
def new_heuristic(graph, start, end):
    
    '''
    We used dijkstra algorithm to find the shortest path and return the sum of the edges as a heuristic value
    and this will never overestimate the actual distance, therefore it is admissible
    '''
    distances = {node: float('inf') for node in graph.vertices}
    distances[start] = 0
    visited = set()
    paths = {start: [start]}
    heap = [(0, start)]

    while heap:
        current_distance, current_node = heapq.heappop(heap)

        if current_node in visited:
            continue

        for neighbor, distance in graph.vertices[current_node].edges:
            total_distance = distances[current_node] + distance
            if total_distance < distances[neighbor.name]:
                distances[neighbor.name] = total_distance
                paths[neighbor.name] = paths[current_node] + [neighbor]
                heapq.heappush(heap, (total_distance, neighbor.name))

        visited.add(current_node)

        if current_node == end:
            return distances[end]

    return None, None


In [ ]:
write_list = []
for i in range(16):
    graph_avg_times = []
    times = 0
    costs = 0
    for pairs in valid_node_pairs[i]:
        node1, node2 = pairs
        time_taken_list = algorithm_runner(aStarSearch, [node1.name, node2.name], 5, graphList[i])
        avg_time = time_taken_list[-2]
        path_cost = time_taken_list[-1]
        graph_avg_times.append((avg_time, path_cost))
    for time, ecost in graph_avg_times:
        times += time
        costs += ecost
    write_list.append("{:.2e}".format(times/len(graph_avg_times)))


In [ ]:
print(write_list)

['2.35e-04', '3.69e-04', '7.68e-04', '7.70e-04', '9.27e-04', '1.86e-03', '2.46e-03', '5.11e-03', '2.18e-03', '4.89e-03', '7.48e-03', '1.05e-02', '5.72e-03', '1.12e-02', '2.25e-02', '2.66e-02']


In [ ]:
# table_writer(write_list,'Report.docx')

<h1>Node centrality rankings</h1>

<h1>Degree centrality</h1>

In [ ]:
def degree_centrality(graph):
    centrality_scores = {}
    for node in graph.vertices.values():
        centrality_scores[node.name] = len(node.edges)
    return centrality_scores


<h4>Ranker function that returns top ten cities based on their centrality</h4>

In [ ]:
def rank(dic):
    sorted_values = sorted(dic.items(), key=lambda x: x[1], reverse=True)
    top_nodes = []
    for node, centrality in sorted_values[:10]:
        top_nodes.append((node, centrality))
    return top_nodes


In [ ]:
rank(degree_centrality(graph))

[('Sibiu', 4),
 ('Bucharest', 4),
 ('Arad', 3),
 ('Craiova', 3),
 ('Rimnicu', 3),
 ('Pitesti', 3),
 ('Urziceni', 3),
 ('Oradea', 2),
 ('Zerind', 2),
 ('Timisoara', 2)]

<h1>Closeness Centrality</h1>

In [ ]:
import queue

def closeness_centrality(graph):
    centrality = {}
    for node in graph.vertices.values():
        visited = set()
        q = queue.Queue()
        q.put((node, 0))
        visited.add(node)
        total_distance = 0
        while not q.empty():
            curr_node, distance = q.get()
            total_distance += distance
            for neighbor, weight in curr_node.edges:
                if neighbor not in visited:
                    visited.add(neighbor)
                    q.put((neighbor, distance + weight))
        centrality[node.name] = (len(visited) - 1) / total_distance if total_distance != 0 else 0
    return centrality


In [ ]:
rank(closeness_centrality(graph))

[('Pitesti', 0.0034025787965616047),
 ('Rimnicu', 0.003269098417068135),
 ('Bucharest', 0.0031040679627511846),
 ('Craiova', 0.002935269581337865),
 ('Sibiu', 0.0028936947913493754),
 ('Fagaras', 0.0027941176470588237),
 ('Urziceni', 0.0027937068078223793),
 ('Drobeta', 0.0024733142410830514),
 ('Giurgiu', 0.0024544632476424235),
 ('Arad', 0.002431533145636038)]

<h1>Eigenvector Centrality </h1>

In [ ]:
import math

def eigen_vector_centrality(graph, max_iterations=100, tolerance=1.0e-6):
    centrality = {node.name: 1.0 for node in graph.vertices.values()}
    for _ in range(max_iterations):
        previous_centrality = centrality.copy()
        for node in graph.vertices.values():
            node_centrality = 0.0
            for neighbor, weight in node.edges:
                node_centrality += previous_centrality[neighbor.name] * weight
            centrality[node.name] = node_centrality
        norm = math.sqrt(sum(value**2 for value in centrality.values()))
        if norm == 0.0:
            norm = 1.0
        error = sum(abs(centrality[node.name]/norm - previous_centrality[node.name]/norm) for node in graph.vertices.values())
        if error < tolerance:
            break
        for node in graph.vertices.values():
            centrality[node.name] /= norm
    return centrality


<h1>PageRank Centrality</h1>

In [ ]:
import queue

def pagerank_centrality(graph, damping_factor=0.85, max_iterations=100, convergence_threshold=1e-4):
    node_count = len(graph.vertices)
    page_rank = {node.name: 1 / node_count for node in graph.vertices.values()}
    out_degree = {node.name: len(node.edges) for node in graph.vertices.values()}
    in_links = {}
    for node in graph.vertices.values():
        for neighbor, _ in node.edges:
            in_links[neighbor.name] = in_links.get(neighbor.name, []) + [node.name]
    
    for i in range(max_iterations):
        prev_page_rank = page_rank.copy()
        delta = 0
        for node in graph.vertices.values():
            rank_sum = 0
            for in_link in in_links.get(node.name, []):
                rank_sum += prev_page_rank[in_link] / out_degree[in_link]
            page_rank[node.name] = (1 - damping_factor) / node_count + damping_factor * rank_sum
            delta += abs(page_rank[node.name] - prev_page_rank[node.name])
        if delta < convergence_threshold:
            break
    
    return page_rank


<h1>Betweenness centrality</h1>

<h4>It uses the famous Brandes algorithm to compute betweenness centrality</h4>

In [ ]:
def betweenness_centrality(graph):
    centrality = {node.name: 0 for node in graph.vertices.values()}
    for node in graph.vertices.values():
        stack = []
        predecessors = {node.name: [] for node in graph.vertices.values()}
        num_shortest_paths = {node.name: 0 for node in graph.vertices.values()}
        distance = {node.name: -1 for node in graph.vertices.values()}
        distance[node.name] = 0
        num_shortest_paths[node.name] = 1
        queue = [node]
        while queue:
            current_node = queue.pop(0)
            stack.append(current_node)
            for neighbor, weight in current_node.edges:
                if distance[neighbor.name] == -1:
                    queue.append(neighbor)
                    distance[neighbor.name] = distance[current_node.name] + weight
                if distance[neighbor.name] == distance[current_node.name] + weight:
                    num_shortest_paths[neighbor.name] += num_shortest_paths[current_node.name]
                    predecessors[neighbor.name].append(current_node)
        credit = {node.name: 1 for node in graph.vertices.values()}
        while stack:
            current_node = stack.pop()
            for predecessor in predecessors[current_node.name]:
                credit[predecessor.name] += credit[current_node.name] * num_shortest_paths[predecessor.name] / num_shortest_paths[current_node.name]
            if current_node != node:
                centrality[current_node.name] += credit[current_node.name]
    return centrality


In [ ]:
clist = []
vlist = []
for city, val in rank(betweenness_centrality(graph)):
    clist.append(city)
    vlist.append(round(val, 5))
    

In [ ]:
# table_writer(vlist,'Report.docx')